In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM, NHITS
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../data/cleaned_data/average_monthly_rating_messenger.csv')
df['Month-Year'] = pd.to_datetime(df['Month-Year'], format='%Y-%m')
df.rename(columns={'Month-Year': 'ds', 'averageRating': 'y'}, inplace=True)
df['ds'] = pd.to_datetime(df['ds'])
df['unique_id'] = 0

Y_df = df 
Y_df.head()

In [ ]:
import numpy as np
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS, LSTM
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calculate the split point (80% for training)
split_point = int(len(Y_df) * 0.8)
Y_train = Y_df.iloc[:split_point].copy()
Y_test = Y_df.iloc[split_point:].copy()

# Set horizon to match test set size
horizon = len(Y_test)
print(f"\nHorizon length: {horizon}")

# Define models
models = [
    LSTM(
        h=horizon,
        max_steps=500,
        scaler_type='standard',
        encoder_hidden_size=64,
        decoder_hidden_size=64,
    ),
    NHITS(
        h=horizon,
        input_size=2 * horizon,
        max_steps=100,
        n_freq_downsample=[2, 1, 1]
    )
]

In [ ]:
# Train models on training data
nf = NeuralForecast(models=models, freq='M')
nf.fit(df=Y_train)

# Generate predictions for test period
Y_hat_df = nf.predict(Y_train).reset_index(drop=True)

# Calculate metrics and weights for ensemble
eval_results = {}
model_weights = {}

for model in ['LSTM', 'NHITS']:
    eval_df = pd.DataFrame({
        'ds': Y_test['ds'],
        'actual': Y_test['y'],
        'predicted': Y_hat_df[model].values[:len(Y_test)]
    })
    
    rmse = np.sqrt(mean_squared_error(eval_df['actual'], eval_df['predicted']))
    mae = mean_absolute_error(eval_df['actual'], eval_df['predicted'])
    mape = np.mean(np.abs((eval_df['actual'] - eval_df['predicted']) / eval_df['actual'])) * 100
    
    eval_results[model] = {
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape
    }
    
    # Calculate weight as inverse of RMSE
    model_weights[model] = 1/rmse

# Normalize weights to sum to 1
weight_sum = sum(model_weights.values())
for model in model_weights:
    model_weights[model] /= weight_sum

print("\nModel Weights:")
for model, weight in model_weights.items():
    print(f"{model}: {weight:.3f}")

# Create and train ensemble model for future predictions
nf_future = NeuralForecast(models=models, freq='M')
nf_future.fit(df=Y_df)
Y_hat_future = nf_future.predict().reset_index(drop=True)